# Dolby.io Analyze Audio and Analyze Speech
PyData Global 2021 | @BradenRiggs1

Learn More about Analyze Audio: https://docs.dolby.io/media-apis/docs/analyze-api-guide

Learn More about Analyze Speech: https://docs.dolby.io/media-apis/docs/speech-analytics-api-guide

---

### Import and Keys

In [ ]:
import os
import requests
import time
import shutil
import json

api_key = json.load(open('key.json'))["key"]

in_path = "raw_audio/"
out_path_audio = "raw_data_audio/"
out_path_speech = "raw_data_speech/"

### Upload Raw Audio

In [ ]:
input_url = "https://api.dolby.com/media/input"
headers = {"x-api-key": api_key, "Content-Type": "application/json", "Accept": "application/json"}

list_of_files = os.listdir(in_path)
list_of_blobs = []

for raw_audio_file in list_of_files:
    blob = "dlb://in/" + raw_audio_file
    list_of_blobs.append(blob)
    body = {"url": blob}

    response = requests.post(input_url, json=body, headers=headers)
    response.raise_for_status()
    data = response.json()
    presigned_url = data["url"]

    print("Uploading {0}".format(in_path + raw_audio_file))
    with open(in_path + raw_audio_file, "rb") as input_file:
        requests.put(presigned_url, data=input_file)

### Analyze Audio for Raw Audio

In [ ]:
url = "https://api.dolby.com/media/analyze"
headers = {"x-api-key": api_key,"Content-Type": "application/json","Accept": "application/json"}
jobIDs = []
list_out_blobs = []

for blob in list_of_blobs:
    out_blob = blob.replace("/in/","/out/").replace(".wav",".json")
    list_out_blobs.append(out_blob)
    body = {"input" : blob ,"output" : out_blob}

    response = requests.post(url, json=body, headers=headers)
    response.raise_for_status()
    jobIDs.append(response.json()["job_id"])


### Download Analyze Audio Results

In [ ]:
url_analyze = "https://api.dolby.com/media/analyze"
url_out = "https://api.dolby.com/media/output"
headers = {"x-api-key": api_key,"Content-Type": "application/json","Accept": "application/json"}
job_out = dict(zip(jobIDs,list_out_blobs))

while job_out:
    for job, output in job_out.items():
        params = {"job_id": job}
        response = requests.get(url_analyze, params=params, headers=headers)
        response.raise_for_status()
        
        if response.json()["status"] == 'Success':
            args = {"url": output}
            
            with requests.get(url_out, params=args, headers=headers, stream=True) as response:
                response.raise_for_status()
                response.raw.decode_content = True
                
                print("Downloading into {0}".format(out_path_audio + output.split("/")[-1]))
                with open(out_path_audio +output.split("/")[-1], "wb") as output_file:
                    shutil.copyfileobj(response.raw, output_file)
                    
            del job_out[job]
            break

### Analyze Speech for Raw Audio

In [ ]:
url = "https://api.dolby.com/media/analyze/speech"
headers = {"x-api-key": api_key,"Content-Type": "application/json","Accept": "application/json"}
jobIDs = []
list_out_blobs = []

for blob in list_of_blobs:
    out_blob = blob.replace("/in/","/out/").replace(".mp3",".json")
    list_out_blobs.append(out_blob)
    body = {"input" : blob ,"output" : out_blob}

    response = requests.post(url, json=body, headers=headers)
    response.raise_for_status()
    jobIDs.append(response.json()["job_id"])

### Download Analyze Speech Results

In [ ]:
url_analyze = "https://api.dolby.com/media/analyze/speech"
url_out = "https://api.dolby.com/media/output"
headers = {"x-api-key": api_key,"Content-Type": "application/json","Accept": "application/json"}
job_out = dict(zip(jobIDs,list_out_blobs))

while job_out:
    for job, output in job_out.items():
        params = {"job_id": job}
        response = requests.get(url_analyze, params=params, headers=headers)
        response.raise_for_status()
        
        if response.json()["status"] == 'Success':
            args = {"url": output}
            
            with requests.get(url_out, params=args, headers=headers, stream=True) as response:
                response.raise_for_status()
                response.raw.decode_content = True
                
                print("Downloading into {0}".format(out_path_speech + output.split("/")[-1]))
                with open(out_path_speech +output.split("/")[-1], "wb") as output_file:
                    shutil.copyfileobj(response.raw, output_file)
                    
            del job_out[job]
            break

Thats it, you've created two JSON files for each of your original audio files! Check out the flatten notebook for converting the data into a CSV

## Like this project and have some ideas of your own?
---

You can submit your ideas here:https://go.dolby.io/pydata-2021

The Dolby.io team will be picking their **5 favorite Analyze API ideas live during the workshop** and an **additional 5 favorite Analyze API ideas after November 7th** so add some of your favorite ideas for how you can use our API and win some free API minutes.